# 初期設定

# Tobii pro nano データ取得プログラム

# 5月24, 25の作業内容記載
---

# Tobii Pro SDKのインストール

In [1]:
!pip install tobii_research

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


# ライブラリのインストール

In [1]:
#ライブラリのインポート
import tobii_research as tr
import time
import pandas as pd

# インテリセンスの設定

In [2]:
%config IPCompleter.greedy =True

---
# データの収集

## Tobiiの設定

In [3]:
#tobiiを見つける
found_eyetrackers = tr.find_all_eyetrackers()

## 接続されているTobiiの詳細

In [4]:
#my_eyetracerに一つ目のtobiiのデータが入る
#一つ目のtobiiのデータを出力する
my_eyetracker = found_eyetrackers[0]
print("Address: " + my_eyetracker.address)
print("Model: " + my_eyetracker.model)
print("Name (It's OK if this is empty): " + my_eyetracker.device_name)
print("Serial number: " + my_eyetracker.serial_number)

Address: tobii-prp://TPNA1-030251450253
Model: Tobii Pro Nano
Name (It's OK if this is empty): 
Serial number: TPNA1-030251450253


## データのカラム名を定義

In [1]:
#瞳孔と視線データを格納するデータフレームを定義
#[左目の瞳孔の大きさ, 右目の瞳孔の大きさ, 左目の視線座標, 右目の視線座標]

col_names=['L_pupil','R_pupil','L_gaze','R_gaze']

## キャリブレーションプログラム
キャリブレーションはソフトで実行できるため、このコードは使用しなくて良い。  
手動実行のために記載しているが使用は非推奨

In [6]:
# #この辺でキャリブレーション
# calibration = tr.ScreenBasedCalibration(my_eyetracker)

# #キャリブレーションの開始
# calibration.enter_calibration_mode()

# points_to_calibrate = [(0.5, 0.5), (0.1, 0.1),
#                            (0.1, 0.9), (0.9, 0.1), (0.9, 0.9)]

# for point in points_to_calibrate:
#     print(("画面に点が表示されます {0}.").format(point))
#     draw_caribration_points(point)
           
    
#     print(("Collecting data at {0}.").format(point))
        
#     if calibration.collect_data(point[0], point[1]) != tr.CALIBRATION_STATUS_SUCCESS:
#         calibration.collect_data(point[0], point[1])
        
#     print('Computing and applying calibration.')
#     calibration_result = calibration.compute_and_apply()
#     print(('Compute and apply returned {0} and collected at {1} points.').
#           format(calibration_result.status, len(calibration_result.calibration_points)))
#     calibration.leave_calibration_mode()
           

## キャリブレーションの終了

In [7]:
#calibration.leave_calibration_mode()

## 確認用

In [8]:
# #視線データを数字で出力する
# #左目の情報と右目の情報がそれぞれ出力される
# def gaze_data_callback(gaze_data):
#     # Print gaze points of left and right eye
#     print("Left eye: ({gaze_left_eye}) \t Right eye: ({gaze_right_eye})".format(
#         gaze_left_eye=gaze_data['left_gaze_point_on_display_area'],
#         gaze_right_eye=gaze_data['right_gaze_point_on_display_area']))

# 収集したデータを格納するためのデータフレームを定義

In [9]:
df = pd.DataFrame(columns=col_names)

# Tobiiから得たデータをデータフレームに格納するための関数

In [10]:
#視線データを数字で出力する
#左目の情報と右目の情報がそれぞれ出力される

def gaze_data_callback(gaze_data):
    #gaze_dataに瞳孔や視線データなどのデータが様々入っているのでその中から指定する
    
    left_pupil=gaze_data['left_pupil_diameter']
    left_gaze=gaze_data['left_gaze_point_on_display_area']
    right_pupil=gaze_data['right_pupil_diameter']
    right_gaze=gaze_data['right_gaze_point_on_display_area']
    
    eye_data=[left_pupil,right_pupil,left_gaze,right_gaze]

    temp = pd.Series([left_pupil,right_pupil,left_gaze,right_gaze],index=col_names)
    
    global df
    
    df=df.append(temp,ignore_index=True)

          
#先にカラムを作成し、追加をしていく
#pandasを使ってpdに変換しつつ

# データ収集開始

In [11]:
#つないでいるtobiiに紐づけをしてデータ
#第一引数:定数
#第二引数:定義した関数
#第三引数:辞書として保存することを指定
my_eyetracker.subscribe_to(tr.EYETRACKER_GAZE_DATA, gaze_data_callback, as_dictionary=True)

# データ収集終了前の処理

In [12]:
#実行することで処理を5秒止める
time.sleep(5)
#注意してくだいさい
#最後が取れなくなる可能性あり

KeyboardInterrupt: 

# データ収集終了

In [13]:
#登録の解除をしてデータの収集を終える
my_eyetracker.unsubscribe_from(tr.EYETRACKER_GAZE_DATA, gaze_data_callback)

# 取り出したデータをCSVファイルへ出力し保存

In [16]:
df.to_csv('csv_data/test-pupil_data.csv',index=False,header=True,mode = 'a')

# CSVデータの確認

In [17]:
test_df=pd.read_csv('csv_data/test-pupil_data.csv')

In [18]:
test_df.describe

<bound method NDFrame.describe of         L_pupil   R_pupil                                    L_gaze  \
0      2.928940  2.912064  (0.4219726622104645, 0.5492538809776306)   
1      2.939255  2.908813  (0.4220326542854309, 0.5475883483886719)   
2      2.946365  2.901596  (0.4177680015563965, 0.5460090041160583)   
3      2.939331  2.908386   (0.4238233268260956, 0.553229570388794)   
4      2.936218  2.900009  (0.4275253713130951, 0.5531033277511597)   
...         ...       ...                                       ...   
51426       NaN       NaN                                (nan, nan)   
51427       NaN       NaN                                (nan, nan)   
51428       NaN       NaN                                (nan, nan)   
51429       NaN       NaN                                (nan, nan)   
51430       NaN       NaN                                (nan, nan)   

                                          R_gaze  
0       (0.3982973098754883, 0.5825415849685669)  
1      (0.3

---
## 5/24
## Tobii pro nano 環境構築/Tobii_Data_Collection.ipynbの作成

これまでの進捗として、メルスペクトログラムを二次元のカラー画像から、3次元の立体グラフとして表示する画像やアニメーションを作成した。  
それらは診断結果を提示する際に、患者や非専門医への理解や説得の支援に有効であるのかを調査する必要がある。  
そのために各種診断結果をパソコンの画面上に提示した際に、疾病の根拠などの見て欲しい部分を患者が見ているか、理解ができているかを調べる。  
実験機材としてTobii pro nanoセンサを用いて、パソコンをみる被験者の視線データを収集する。  
またこのセンサは瞳孔の大きさも計測することが可能なためそれらも収集する。  
Tobii pro nanoセンサはセンサを使用するためのSDKと適切なPythonのバージョンを用意するなどの環境構築が必要である。  
そのためSDKを公式サイトからインストールし、Pythonも実行可能なバージョンである3.5系をインストールし、仮想環境を構築した。  
またTobiiセンサを使用するためのツールの位置設定等の初期設定を実施した。  

Tobiiセンサから視線データを取得し、ファイルに記述するためにはPythonでプログラムを作成する必要がある。  
そのため本プログラムファイルTobii_Data_Collection.ipynbを作成した。  
プログラムではTobiiセンサを扱うためのtobii_researchをインストールし、センサを接続するためのコーディング作業を実施した。  

---
## 5/25
## Tobii_Data_Collection.ipynbの作成/Tobii_Data_Collection.ipynbのテスト実行

前回作成した、Tobii_Data_Collection.ipynbにデータ取得と保存のプログラムを追加するため、引き続きコーディング作業を実施した。  
データ取得は、Tobiiセンサを接続しツールでキャリブレーションを実行し、使用可能にした状態で本プログラムを実行することで可能にする。  
my_eyetracker.subscribe_to(tr.EYETRACKER_GAZE_DATA, gaze_data_callback, as_dictionary=True)を実行することで、  
瞳孔を'left_pupil_diameter'、'right_pupil_diameter'で取得する。  
視線は'left_gaze_point_on_display_area'、'right_gaze_point_on_display_area'で取得する。  
これらで取得したデータをプログラムでデータフレームを作成し、順次それに格納する。  
視線収集が終了した際にそれらのデータを格納したデータフレームをCSV書き込むことで、データを他のプログラムで利用可能にする。  
  
これらのコーディング作業を実施し、実行が正しくされているかを確認するために、自身の視線データを取得する事前テストを実施した。  
テストで生じた不備やバグを修正した後、取得したデータをCSVに保存し、それらを確認するコードを実行した。  
結果を確認したところ視線データや瞳孔のデータを正しく保存できていることを確認した。  